# Importing the libs

In [4]:
import numpy as np
import pandas as pd
import re #substitution or replacement of substrings
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import random
# !pip install contractions
import contractions
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()
tfidf_vectorizer = TfidfVectorizer()

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


# Defined methods

In [5]:
def clean_text(text):
    # \S matches any character that is not a space tab newline
    text = re.sub(r'http\S+', '', text)
    #remove emojis
    text = text.encode('ascii', 'ignore').decode('ascii')
    # igore => specifies how to handle characters that cannot be represented in ASCII
    # remove htmk tags  => . any char except newline
    text = re.sub(r'<.*>', '', text)
    # remove punctiuations
    text = re.sub(r'[^\w\s]', '', text)
    return text

def remove_stopwords(text):
    words = word_tokenize(text)
    english_stopwords = set(stopwords.words('english'))
    filtered_words = []
    for word in words:
        if word.lower() not in english_stopwords:
            filtered_words.append(word)
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def spell_check_and_correction(text):
    blob = TextBlob(text)
    corrected_text = str(blob.correct())
    return corrected_text

def to_lower(s):
    return s.lower()

def expanding_contractions(text):
    text = contractions.fix(text)
    return text

def tokenization_and_lemmatization(text):
    text_tokenized = []
    
    words = nltk.word_tokenize(text) #Tokenize

    for word in words:
        temp = lemmatizer.lemmatize(word) #lemmatize
        text_tokenized.append(porter_stemmer.stem(temp)) #stem        
    
#     text_tokenized = text_tokenized[len(text_tokenized) <= 10]
    
    return text_tokenized

In [6]:
def preprocess_text(text):
    text = clean_text(text)
    text = spell_check_and_correction(text)
    text = expanding_contractions(text)
    text = remove_stopwords(text)
    text = to_lower(text)
    text = tokenization_and_lemmatization(text)

    return text

# Testing the methods
# print(preprocess_text("print they should pay all the back all the money plus interest the entire family and everyone who came in with them need to be deported asap why didn't it take two years to bust them here we go again another group stealing from the government and taxpayers a group of somalis stole over four million in government benefits over just  months weve reported on numerous cases like this one where the muslim refugeesimmigrants commit fraud by scamming our systemits way out of control more related"))

# Data analysis

In [7]:
df_before = pd.read_csv("Before.csv",encoding = "latin1")
df_before.head(10)

,text,label
0,Print They should pay all the back all the mon...,0.0
1,Why Did Attorney General Loretta Lynch Plead T...,0.0
2,Red State : \nFox News Sunday reported this mo...,0.0
3,Email Kayla Mueller was a prisoner and torture...,0.0
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,0.0
5,Print Hillary goes absolutely berserk! She exp...,0.0
6,BREAKING! NYPD Ready To Make Arrests In Weiner...,0.0
7,BREAKING! NYPD Ready To Make Arrests In Weiner...,0.0
8,\nLimbaugh said that the revelations in the Wi...,0.0
9,Email \nThese people are sick and evil. They w...,0.0


In [8]:
df_before.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156790 entries, 0 to 156789
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    153247 non-null  object 
 1   label   151791 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.4+ MB


In [9]:
df_before["label"].value_counts()

label
0.0    84308
1.0    67483
Name: count, dtype: int64

In [10]:
plt.pie(df_before["label"])

ValueError: cannot convert float NaN to integer

ValueError: need at least one array to concatenate

<Figure size 640x480 with 1 Axes>

In [7]:
df_before.isna().sum()

text     3543
label    4999
dtype: int64

In [8]:
# remove nan and -inf values
df_after = df_before.dropna()
df_after.isna().sum()

text     0
label    0
dtype: int64

In [9]:
# change the type of the label column to int
df_after['label'] = df_after['label'].astype(int)
df_after

C:\Users\alihi\AppData\Local\Temp\ipykernel_10324\3972895233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after['label'] = df_after['label'].astype(int)


,text,label
0,Print They should pay all the back all the mon...,0
1,Why Did Attorney General Loretta Lynch Plead T...,0
2,Red State : \nFox News Sunday reported this mo...,0
3,Email Kayla Mueller was a prisoner and torture...,0
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,0
...,...,...
156784,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
156785,"LONDON (Reuters) - LexisNexis, a provider of l...",1
156786,MINSK (Reuters) - In the shadow of disused Sov...,1
156787,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [10]:
df_after.duplicated().sum()

12963

In [11]:
df_after = df_after.drop_duplicates()
df_after

,text,label
0,Print They should pay all the back all the mon...,0
1,Why Did Attorney General Loretta Lynch Plead T...,0
2,Red State : \nFox News Sunday reported this mo...,0
3,Email Kayla Mueller was a prisoner and torture...,0
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,0
...,...,...
156784,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
156785,"LONDON (Reuters) - LexisNexis, a provider of l...",1
156786,MINSK (Reuters) - In the shadow of disused Sov...,1
156787,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [12]:
# shuffle the rows
# df_after = df_after.sample(frac=1)
# df_after

# Data preprocssing

In [13]:
df_after.loc[0:16,"text"] = df_after.loc[0:16,"text"].apply(preprocess_text)

In [14]:
display(df_after.loc[0:16])
display(df_before.loc[0:16])

,text,label
0,"[print, pay, back, money, plu, interest, entir...",0
1,"[attorney, gener, biretta, lynch, lead, fifth,...",0
2,"[red, state, box, news, sunday, report, morn, ...",0
3,"[email, paula, fuller, prison, tortur, isi, ch...",0
4,"[email, healthcar, reform, make, america, grea...",0
5,"[print, axillari, go, absolut, verser, explod,...",0
6,"[break, nypd, readi, take, arrest, rein, caseh...",0
7,"[break, nypd, readi, take, arrest, rein, caseh...",0
8,"[limbaugh, said, revel, wikileak, materi, star...",0
9,"[email, peopl, sick, evil, stop, noth, get, wa...",0


,text,label
0,Print They should pay all the back all the mon...,0.0
1,Why Did Attorney General Loretta Lynch Plead T...,0.0
2,Red State : \nFox News Sunday reported this mo...,0.0
3,Email Kayla Mueller was a prisoner and torture...,0.0
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,0.0
5,Print Hillary goes absolutely berserk! She exp...,0.0
6,BREAKING! NYPD Ready To Make Arrests In Weiner...,0.0
7,BREAKING! NYPD Ready To Make Arrests In Weiner...,0.0
8,\nLimbaugh said that the revelations in the Wi...,0.0
9,Email \nThese people are sick and evil. They w...,0.0
